<a href="https://colab.research.google.com/github/mauriciodev/spatialgeodesy/blob/main/exercises/Ex2_Broadcast_orbits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2 - Compute satellite coordinates from the navigation file

In this exercise we are going to download a navigation RINEX file, open it with the Georinex library and compute the coordinates of one satellite at a given time. The files are obtained from BKG, one of the International GNSS Service (IGS) data centers.

# Preparation

## Installing dependencies

In [2]:
#Python standard libraries
import os #File path operations.
import shutil #Shell operations. Unzipping, moving files, etc.
import urllib.request #Downloader.

#External libraries
import numpy as np #Numeric Python.
import pandas as pd #Python Data Analysis Library.
import matplotlib.pyplot as plt #Plots.
import georinex #RINEX files reader.
import xarray as xa #Multi dimension arrays. For georinex.   

In [1]:
#Installing the package that reads rinex and sp3 files
!pip install --upgrade pip
!pip install git+https://github.com/geospace-code/georinex
import georinex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/geospace-code/georinex to /tmp/pip-req-build-sl3w8a_l
  Running command git clone --filter=blob:none --quiet https://github.com/geospace-code/georinex /tmp/pip-req-build-sl3w8a_l
  Resolved https://github.com/geospace-code/georinex to commit c689a5a6bc2ffb68bc055f150f1da1b6bab12812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.9/877.9 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.2/89

# Input parameters

In [3]:
#This is the reference time. We are going to use it to download the navigation files and compute the orbital position. 
t = pd.to_datetime('2023-01-01 02:00:01') 

In [4]:
#CHANGE THIS: Use the code from the previous exercises to compute 
year = t.year
dayOfYear = t.day_of_year
twoDigitYear = t.year %100

#File name is based on the Day of Year
#formatiing with previous variables. Note the :03 on each variable to fill with leading zeros.
#https://igs.bkg.bund.de/root_ftp/IGS/BRDC/2022/001/brdc0010.22n.gz
navigation_file_url=f"https://igs.bkg.bund.de/root_ftp/IGS/BRDC/{year}/{dayOfYear:03}/brdc{dayOfYear:03}0.{twoDigitYear}n.gz" 
print(navigation_file_url)

https://igs.bkg.bund.de/root_ftp/IGS/BRDC/2022/001/brdc0010.22n.gz


## Downloading the broadcast Navigation File

In [5]:
#splitting the file name from the URL
navRinexFile=os.path.split(navigation_file_url)[1] 
local_filename, headers = urllib.request.urlretrieve(navigation_file_url, navRinexFile) #download the file saving as the name in zipFile
print(local_filename)

brdc0010.22n.gz


# Showing the first 20 lines of the navigation file

In [99]:
# the ! prefix runs the commands on a linux shell. So we can use both python and shell commands on this notebook.
!uncompress $navRinexFile -k
uncompressed=navRinexFile[:-3]
!head -n 20  $uncompressed

gzip: brdc0010.22n already exists; do you wish to overwrite (y or n)? y
     2.11           N: GPS NAV DATA                         RINEX VERSION / TYPE
teqc  2019Feb25     BKG Frankfurt       20220102 09:17:41UTCPGM / RUN BY / DATE
Linux 2.6.32-573.12.1.x86_64|x86_64|gcc -static|Linux 64|=+ COMMENT
Converto v3.5.6     IGN                 20220102 000533 UTC COMMENT
Linux 2.6.32-573.12.1.x86_64|x86_64|gcc|Linux 64|=+         COMMENT
Concatenated RINEX files (35)                               COMMENT
                                                            END OF HEADER
 1 22  1  1  0  0  0.0 4.691267386079D-04-1.000444171950D-11 0.000000000000D+00
    3.900000000000D+01-1.411250000000D+02 3.988380417768D-09-6.242942382352D-01
   -7.363036274910D-06 1.121813920327D-02 4.695728421211D-06 5.153674995422D+03
    5.184000000000D+05-3.166496753693D-08-1.036611240093D+00 1.955777406693D-07
    9.864187694897D-01 2.997500000000D+02 8.840876015687D-01-8.133553080847D-09
   -3.778728827795D-1

# Opening the navigation file with Georinex

In [6]:
nav = georinex.rinexnav(navRinexFile)
#Let's see the data that was read
nav

<xarray.Dataset>
Dimensions:           (time: 31, sv: 32)
Coordinates:
  * time              (time) datetime64[ns] 2022-01-01 ... 2022-01-01T23:59:44
  * sv                (sv) <U3 'G01' 'G02' 'G03' 'G04' ... 'G30' 'G31' 'G32'
Data variables: (12/29)
    SVclockBias       (time, sv) float64 0.0004691 -0.0006474 ... -4.397e-05
    SVclockDrift      (time, sv) float64 -1e-11 -1.137e-12 ... -5.798e-12
    SVclockDriftRate  (time, sv) float64 0.0 0.0 0.0 0.0 0.0 ... nan nan 0.0 0.0
    IODE              (time, sv) float64 39.0 41.0 38.0 222.0 ... nan 15.0 57.0
    Crs               (time, sv) float64 -141.1 -142.7 -59.75 ... -25.53 18.16
    DeltaN            (time, sv) float64 3.988e-09 4.556e-09 ... 4.741e-09
    ...                ...
    SVacc             (time, sv) float64 2.0 2.0 2.0 2.0 2.0 ... nan nan 2.0 2.0
    health            (time, sv) float64 0.0 0.0 0.0 0.0 0.0 ... nan nan 0.0 0.0
    TGD               (time, sv) float64 5.122e-09 -1.77e-08 ... 4.657e-10
    IODC              (time, sv) float64 39.0 41.0 38.0 990.0 ... nan 15.0 57.0
    TransTime         (time, sv) float64 5.112e+05 5.115e+05 ... 6.018e+05
    FitIntvl          (time, sv) float64 4.0 4.0 4.0 4.0 4.0 ... nan nan 4.0 nan
Attributes:
    version:    2.11
    svtype:     ['G']
    rinextype:  nav
    filename:   brdc0010.22n.gz

# Selecting subsets of the navigation file
Here we are selecting the Spatial Vehicle G01 (GPS), at the time previously specified in t. The method "pad" brings the "latest" orbit. Check the time of the orbit that we found on the "Coordinates" section. The orbital parameters are in the "Data variables". 

In [7]:
sat='G01'
orbit=nav.sel(sv=sat,time=t, method="pad")
orbit

<xarray.Dataset>
Dimensions:           ()
Coordinates:
    time              datetime64[ns] 2022-01-01T02:00:00
    sv                <U3 'G01'
Data variables: (12/29)
    SVclockBias       float64 0.0004691
    SVclockDrift      float64 -1e-11
    SVclockDriftRate  float64 0.0
    IODE              float64 70.0
    Crs               float64 -137.8
    DeltaN            float64 4.01e-09
    ...                ...
    SVacc             float64 2.0
    health            float64 0.0
    TGD               float64 5.122e-09
    IODC              float64 70.0
    TransTime         float64 5.184e+05
    FitIntvl          float64 4.0
Attributes:
    version:    2.11
    svtype:     ['G']
    rinextype:  nav
    filename:   brdc0010.22n.gz

# Implement the orbit calculation.
Now follow the formulas on the slides to implement the orbit calculation using the information of G01 at the given time t. Note that t is not the beginning of the orbit period and Toe (time of ephemeris) is counted in seconds of the GPS week.

In [11]:
from numpy import sin, cos, arctan, sqrt #useful math functions
#CHANGE THIS: orbit calculation is unfinished. If you 
earthrate = 7.2921151467e-5 #This is default bOMEGAE84
bGM84 = 3.986005e14 #GM

A=orbit.sqrtA**2 #the nav file provided the square root instead of A

print("A:",A.to_numpy())

print("Satellite position: ",satX,satY,satZ)

A: 26560375.513063643


NameError: ignored

In [8]:
#GPS time of week. You can use your own function from last exercise, if you prefer.
gps_t0=pd.to_datetime('1980-01-06 00:00:00')
deltaDate=t - gps_t0
dayOfWeek= deltaDate.days % 7
t_midnight=t.replace(hour=0, minute=0, second=0)
secondsOfGPSWeek=(t-t_midnight).seconds+dayOfWeek*24*60*60
print("t in seconds of GPS week", secondsOfGPSWeek, "TOE:", orbit.Toe.to_numpy())

t in seconds of GPS week 525601 TOE: 525600.0


# Orbit calculation with Georinex's function

In [9]:
orbit['time']=[t] #We are now chaning the "time" dimension so that the orbit is calculated on a different time.
orbit

<xarray.Dataset>
Dimensions:           (time: 1)
Coordinates:
  * time              (time) datetime64[ns] 2022-01-01T02:00:01
    sv                <U3 'G01'
Data variables: (12/29)
    SVclockBias       float64 0.0004691
    SVclockDrift      float64 -1e-11
    SVclockDriftRate  float64 0.0
    IODE              float64 70.0
    Crs               float64 -137.8
    DeltaN            float64 4.01e-09
    ...                ...
    SVacc             float64 2.0
    health            float64 0.0
    TGD               float64 5.122e-09
    IODC              float64 70.0
    TransTime         float64 5.184e+05
    FitIntvl          float64 4.0
Attributes:
    version:    2.11
    svtype:     ['G']
    rinextype:  nav
    filename:   brdc0010.22n.gz

In [10]:
#Check the code on the link to understand a bit how georinex compute the keplerian orbit
#https://github.com/geospace-code/georinex/blob/main/src/georinex/keplerian.py 
x, y, z = georinex.keplerian2ecef(orbit)
print(x,y,z)

[13370507.39103044] [-7830151.13620469] [21237814.19942809]


# How far are your coordinates from Georinex's orbit?